In [1]:
import pickle

with open('./datasets/small/HAS_e.pickle', 'rb') as f:
    HAS_e = pickle.load(f)

In [2]:
from utils import load_ldas, load_bertopics

ldas = load_ldas()
bertopics = load_bertopics()

d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython'

In [3]:
evaluation_time_lda = []
evaluation_time_bertopic = []

In [4]:
from time import time
from utils import get_topics_lda, get_topics_bertopic, get_coherence, get_diversity

with open('./datasets/small/HAS_pt.pickle', 'rb') as f:
    HAS_pt = pickle.load(f)

def evaluate_models(models, HAS_e, lda=False):
    r = [] # result
    for v in models:
        if lda:
            topics = get_topics_lda(models[v]['model'], HAS_e[v]['id2word'])
        else:
            topics = get_topics_bertopic(bertopics[v]['model'])
        t_c = time()
        c = get_coherence(
            topics=topics,
            texts=HAS_e[v]['T'],
            dictionary=HAS_e[v]['id2word']
        )
        t_d = time()
        d = get_diversity(topics)
        t_end = time()
        if lda:
            evaluation_time_lda.append({
                'variant': v,
                'preprocessing': HAS_pt[v],
                'corpus': HAS_e[v]['time'],
                'training': models[v]['time'],
                'coherence': t_d - t_c,
                'diversity': t_end - t_d,
            })
        else:
            evaluation_time_bertopic.append({
                'variant': v,
                'preprocessing': HAS_pt[v],
                'corpus': HAS_e[v]['time'],
                'training': models[v]['time'],
                'coherence': t_d - t_c,
                'diversity': t_end - t_d,
            })
        r.append({
            'variant': v,
            'coherence': c,
            'diversity': d,
            'score': (c + d) / 2 # TODO: use product instead of mean
        })
    return r

In [5]:
import pandas as pd

evaluation_lda = pd.DataFrame(evaluate_models(ldas, HAS_e, lda=True))
evaluation_bertopic = pd.DataFrame(evaluate_models(bertopics, HAS_e))

In [6]:
evaluation_lda

,variant,coherence,diversity,score
0,HCLWN,0.374301,0.875,0.62465


In [7]:
evaluation_bertopic

,variant,coherence,diversity,score
0,HCLWN,0.468636,0.95,0.709318


In [8]:
pd.DataFrame(evaluation_time_lda)

,variant,preprocessing,corpus,training,coherence,diversity
0,HCLWN,3.049255,0.005002,16.398674,2.055992,0.0


In [9]:
pd.DataFrame(evaluation_time_bertopic)

,variant,preprocessing,corpus,training,coherence,diversity
0,HCLWN,3.049255,0.005002,21.885665,1.905481,0.0
